In [2]:
import os

In [3]:
%pwd


'c:\\ml step\\endtoendmlops\\Flask-E2E-Mlops\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\ml step\\endtoendmlops\\Flask-E2E-Mlops'

In [5]:
import pandas as pd

data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [7]:
data.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [8]:
for d in data.dtypes:
    print(d)

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64


In [9]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [10]:
data.shape

(1599, 12)

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema:dict

In [12]:
from src.FlaskMlops.constants import *
from src.FlaskMlops.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifact_root])
    
    def get_data_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config
        
    

In [14]:
import os
from src.FlaskMlops import logger

In [17]:
class Datavalidation:
    def __init__(self,config:DataValidationConfig):
        self.config=config
    
    def validate_all_columns(self)-> bool:
        try:
            validation_status=None
            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols=list(data.columns)
            datatypes_columns=list(data.dtypes)
            
            all_schema=self.config.all_schema.keys()
            datatypes=self.config.all_schema.values()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"Validation status:{validation_status}")
                
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"Validation status: {validation_status}")
                
                
            return validation_status
        
        except Exception as e:
            raise e
            
    
    
        

In [18]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=Datavalidation(config=data_validation_config)
    data_validation.validate_all_columns()
    
except Exception as e:
    raise e
    

[2025-12-24 14:22:22,747: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2025-12-24 14:22:22,765: INFO: common: Yaml file: params.yaml loaded successfully]
[2025-12-24 14:22:22,771: INFO: common: Yaml file: schema.yaml loaded successfully]
[2025-12-24 14:22:22,779: INFO: common: Created directory at: artifacts]


[2025-12-24 14:22:22,785: INFO: common: Created directory at: artifacts/data_validation]


In [ ]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [ ]:
all_col=list(data.columns)
for col in all_col:
    print(data[col].dtypes)

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64


In [ ]:
s=data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [ ]:
s=read_yaml(SCHEMA_FILE_PATH)

[2025-12-24 00:33:26,877: INFO: common: Yaml file: schema.yaml loaded successfully]


In [ ]:
dict(map(lambda x: (x[0].replace(" ", "_"), x[1]), s.COLUMNS.items()))


{'fixed_acidity': 'float64',
 'volatile_acidity': 'float64',
 'citric_acid': 'float64',
 'residual_sugar': 'float64',
 'chlorides': 'float64',
 'free_sulfur_dioxide': 'float64',
 'total_sulfur_dioxide': 'float64',
 'density': 'float64',
 'pH': 'float64',
 'sulphates': 'float64',
 'alcohol': 'float64',
 'quality': 'int64'}

In [ ]:
ss=[col[0].replace(" ","_") for col in s.COLUMNS]

In [ ]:
ss

['f', 'v', 'c', 'r', 'c', 'f', 't', 'd', 'p', 's', 'a', 'q']